<a href="https://colab.research.google.com/github/sandeep92134/The-Data-Science-Workshop-By-Packt/blob/master/module%2013/Activity13.01%3A%20Finding%20the%20Best%20Balancing%20Technique%20by%20Fitting%20a%20Classifier%20on%20the%20Telecom%20Churn%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install smote-variants

     |████████████████████████████████| 143kB 8.1MB/s 
  Created wheel for statistics: filename=statistics-1.0.3.5-cp36-none-any.whl size=7454 sha256=c54ee027c7d8d861ed08d8fd84b0b14cff18fef0e267880f9531446add72d636
  Stored in directory: /root/.cache/pip/wheels/75/55/90/73aa7662bfb4565b567618547a275f01372a678ca92ecd64f3
  Created wheel for minisom: filename=MiniSom-2.2.7-cp36-none-any.whl size=8607 sha256=bbb12650008bac58c6bbc7049ca5f2c42d09c6c6f06803647b4ddeddf0918322
  Stored in directory: /root/.cache/pip/wheels/db/9a/af/0fca4ced67fec66c00998c4bf55b13f0a4c882ee775f278862
Successfully built statistics minisom


In [1]:
# Loading data from the Github repository
import pandas as pd
filename = 'https://raw.githubusercontent.com/sandeep92134/The-Data-Science-Workshop-By-Packt/master/module%2013/datasets/churn.csv'
# Loading the data using pandas

churnData = pd.read_csv(filename,sep=",")
churnData.head()

,churn,accountlength,internationalplan,voicemailplan,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls
0,No,128,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,No,107,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,No,137,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,No,84,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,No,75,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


**Feature engineering steps**

Let us now do some feature engineering to the data. First we will scale the numerical data and then convert the ordinal data to 
dummy data

In [2]:
# Normalising data
from sklearn import preprocessing

minmaxScaler = preprocessing.MinMaxScaler()

# Converting each of the columns to scaled version

churnData['alScaled'] = minmaxScaler.fit_transform(churnData['accountlength'].values.reshape(-1,1))
churnData['nvmmScaled'] = minmaxScaler.fit_transform(churnData['numbervmailmessages'].values.reshape(-1,1))
churnData['tdmScaled'] = minmaxScaler.fit_transform(churnData['totaldayminutes'].values.reshape(-1,1))
churnData['tdcScaled'] = minmaxScaler.fit_transform(churnData['totaldaycalls'].values.reshape(-1,1))
churnData['tdchScaled'] = minmaxScaler.fit_transform(churnData['totaldaycharge'].values.reshape(-1,1))
churnData['temScaled'] = minmaxScaler.fit_transform(churnData['totaleveminutes'].values.reshape(-1,1))
churnData['tecScaled'] = minmaxScaler.fit_transform(churnData['totalevecalls'].values.reshape(-1,1))
churnData['techScaled'] = minmaxScaler.fit_transform(churnData['totalevecharge'].values.reshape(-1,1))
churnData['tnmScaled'] = minmaxScaler.fit_transform(churnData['totalnightminutes'].values.reshape(-1,1))
churnData['tncScaled'] = minmaxScaler.fit_transform(churnData['totalnightcalls'].values.reshape(-1,1))
churnData['tnchScaled'] = minmaxScaler.fit_transform(churnData['totalnightcharge'].values.reshape(-1,1))
churnData['timScaled'] = minmaxScaler.fit_transform(churnData['totalintlminutes'].values.reshape(-1,1))
churnData['ticScaled'] = minmaxScaler.fit_transform(churnData['totalintlcalls'].values.reshape(-1,1))
churnData['tichScaled'] = minmaxScaler.fit_transform(churnData['totalintlcharge'].values.reshape(-1,1))
churnData['ncscScaled'] = minmaxScaler.fit_transform(churnData['numbercustomerservicecalls'].values.reshape(-1,1))




In [3]:
# Dropping the original columns

churnData.drop(['accountlength','numbervmailmessages','totaldayminutes','totaldaycalls','totaldaycharge','totaleveminutes','totalevecalls','totalevecharge','totalnightminutes','totalnightcalls','totalnightcharge','totalintlminutes','totalintlcalls','totalintlcharge','numbercustomerservicecalls'], axis=1, inplace=True)

# Print the head of the data

churnData.head()

,churn,internationalplan,voicemailplan,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled
0,No,no,yes,0.524793,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000,0.111111
1,No,no,yes,0.438017,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185,0.111111
2,No,no,no,0.561983,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0.000000
3,No,yes,no,0.342975,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630,0.222222
4,No,yes,no,0.305785,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556,0.333333


In [4]:
# Converting all the categorical variables to dummy variables
churnCat = pd.get_dummies(churnData[['internationalplan','voicemailplan']])

In [5]:
# Seperating the numerical data
churnNum = churnData[['alScaled','nvmmScaled','tdmScaled','tdcScaled','tdchScaled','temScaled','tecScaled','techScaled','tnmScaled','tncScaled','tnchScaled','timScaled','ticScaled','tichScaled','ncscScaled']]
churnNum.shape

(5000, 15)

In [6]:
# Merging with the original data frame
# Preparing the X variables
X = pd.concat([churnCat, churnNum], axis=1)
print(X.shape)
# Preparing the Y variable
Y = churnData['churn']
print(Y.shape)
X.head()

(5000, 19)
(5000,)


,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled
0,1,0,0,1,0.524793,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000,0.111111
1,1,0,0,1,0.438017,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185,0.111111
2,1,0,1,0,0.561983,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0.000000
3,0,1,1,0,0.342975,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630,0.222222
4,0,1,1,0,0.305785,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556,0.333333


In [7]:
from sklearn.model_selection import train_test_split

# Splitting the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=123)

**Undersampling Method.**






In [8]:
# let us first join the train_x and train_y for ease of operation

trainData = pd.concat([X_train,y_train],axis=1)
trainData.head()

,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled,churn
4036,1,0,0,1,0.256198,0.500000,0.609388,0.484848,0.609270,0.695628,0.894118,0.695891,0.395949,0.622857,0.396173,0.515,0.10,0.514815,0.222222,No
2883,1,0,1,0,0.504132,0.000000,0.595733,0.296970,0.595716,0.652736,0.688235,0.652863,0.605570,0.560000,0.605515,0.490,0.55,0.490741,0.111111,No
4162,0,1,1,0,0.012397,0.000000,0.482788,0.581818,0.482764,0.362387,0.552941,0.362342,0.620506,0.491429,0.620709,0.710,0.20,0.709259,0.000000,Yes
4640,1,0,1,0,0.450413,0.000000,0.714936,0.551515,0.714859,0.569700,0.558824,0.569719,0.630380,0.400000,0.630838,0.645,0.10,0.644444,0.111111,Yes
2430,1,0,0,1,0.491736,0.769231,0.364438,0.600000,0.364458,0.681056,0.458824,0.681009,0.505570,0.691429,0.505909,0.780,0.15,0.779630,0.000000,No


In [9]:
# Finding the indexes of the sample data set where the churn  is 'yes'
ind = trainData[trainData['churn']=='Yes'].index
print(len(ind))

490


In [10]:
# Seperate the minority classes
minData = trainData.loc[ind]
print(minData.shape)

(490, 20)


In [11]:
# Finding indexes of majority class
ind1 = trainData[trainData['churn']=='No'].index
print(len(ind1))

3010


In [12]:
# Seperating the majority class
majData = trainData.loc[ind1]
print(majData.shape)
majData.head()

(3010, 20)


,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled,churn
4036,1,0,0,1,0.256198,0.500000,0.609388,0.484848,0.609270,0.695628,0.894118,0.695891,0.395949,0.622857,0.396173,0.515,0.10,0.514815,0.222222,No
2883,1,0,1,0,0.504132,0.000000,0.595733,0.296970,0.595716,0.652736,0.688235,0.652863,0.605570,0.560000,0.605515,0.490,0.55,0.490741,0.111111,No
2430,1,0,0,1,0.491736,0.769231,0.364438,0.600000,0.364458,0.681056,0.458824,0.681009,0.505570,0.691429,0.505909,0.780,0.15,0.779630,0.000000,No
449,1,0,0,1,0.322314,0.403846,0.751920,0.478788,0.751841,0.557602,0.694118,0.557748,0.438987,0.525714,0.438942,0.315,0.15,0.314815,0.444444,No
4179,1,0,1,0,0.578512,0.000000,0.613940,0.478788,0.613956,0.309871,0.500000,0.309932,0.561519,0.622857,0.561621,0.280,0.20,0.279630,0.222222,No


In [13]:
# Take a random sample equal to length of the minority class to make the data set balanced

majSample = majData.sample(n=len(ind),random_state = 123)
print(majSample.shape)
majSample.head()



(490, 20)


,internationalplan_no,internationalplan_yes,voicemailplan_no,voicemailplan_yes,alScaled,nvmmScaled,tdmScaled,tdcScaled,tdchScaled,temScaled,tecScaled,techScaled,tnmScaled,tncScaled,tnchScaled,timScaled,ticScaled,tichScaled,ncscScaled,churn
1807,1,0,1,0,0.450413,0.000000,0.557895,0.624242,0.557898,0.549079,0.723529,0.549013,0.344051,0.405714,0.344401,0.645,0.05,0.644444,0.333333,No
4578,1,0,1,0,0.475207,0.000000,0.244097,0.533333,0.244143,0.318394,0.658824,0.318344,0.495949,0.520000,0.496342,0.550,0.10,0.550000,0.111111,No
355,1,0,1,0,0.123967,0.000000,0.472546,0.636364,0.472557,0.218037,0.547059,0.218052,0.541013,0.560000,0.541362,0.635,0.10,0.635185,0.111111,No
23,1,0,1,0,0.454545,0.000000,0.314083,0.624242,0.314090,0.377509,0.600000,0.377548,0.480000,0.600000,0.480023,0.385,0.30,0.385185,0.222222,No
1541,1,0,0,1,0.194215,0.692308,0.656899,0.557576,0.656794,0.460819,0.711765,0.461016,0.683544,0.497143,0.683737,0.380,0.20,0.379630,0.333333,No


In [14]:
# Concatenating both data sets

balData = pd.concat([minData,majSample],axis = 0)
print('balanced data set shape',balData.shape)

balanced data set shape (980, 20)


In [15]:
# Shuffling the data set

from sklearn.utils import shuffle

balData = shuffle(balData)


In [16]:
# Making the new X_train and y_train

X_trainNew = balData.iloc[:,0:19]
X_trainNew.shape

(980, 19)

In [17]:
# Making the new y_train
y_trainNew = balData['churn']
y_trainNew.shape

(980,)

In [18]:
from sklearn.linear_model import LogisticRegression
# Defining the LogisticRegression function for Undersampling
churnModel1 = LogisticRegression()
churnModel1.fit(X_trainNew, y_trainNew)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**SMOTE**



In [22]:
import smote_variants as sv
import numpy as np

# Instantiating the SMOTE class
oversampler= sv.SMOTE()

In [23]:
# Creating new training set

X_train_smote, y_train_smote = oversampler.sample(np.array(X_train), np.array(y_train))


2021-02-04 11:29:37,965:INFO:SMOTE: Running sampling via ('SMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


In [24]:
# Training the model with Logistic regression model

# Defining the LogisticRegression function

churnModel2 = LogisticRegression()

churnModel2.fit(X_train_smote, y_train_smote)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**MSMOTE**



In [25]:
import smote_variants as sv
import numpy as np
# Instantiating the MSMOTE class
oversampler= sv.MSMOTE()

In [26]:
# Creating new training sets
X_train_msmote, y_train_msmote = oversampler.sample(np.array(X_train), np.array(y_train))

2021-02-04 11:29:50,932:INFO:MSMOTE: Running sampling via ('MSMOTE', "{'proportion': 1.0, 'n_neighbors': 5, 'n_jobs': 1, 'random_state': None}")


In [27]:
# Fitting model

# Training the model with Logistic regression model

# Defining the LogisticRegression function
churnModel3 = LogisticRegression()
churnModel3.fit(X_train_msmote, y_train_msmote)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

**Predictions with all three models**

Let us create separate predictions with each model

In [28]:
# Predicting using Under sampler
pred_us = churnModel1.predict(X_test)

# Predicting using SMOTE
pred_smote = churnModel2.predict(X_test)

# Predicting using MSMOTE
pred_msmote = churnModel3.predict(X_test)


In [29]:
# Printing accuracy of each model

print('Accuracy of Logistic regression model prediction on test set for Random Undersampled data set: {:.2f}'.format(churnModel1.score(X_test, y_test)))

print('Accuracy of Logistic regression model prediction on test set for SMOTE data set: {:.2f}'.format(churnModel2.score(X_test, y_test)))

print('Accuracy of Logistic regression model prediction on test set for MSMOTE data set: {:.2f}'.format(churnModel3.score(X_test, y_test)))

Accuracy of Logistic regression model prediction on test set for Random Undersampled data set: 0.79
Accuracy of Logistic regression model prediction on test set for SMOTE data set: 0.78
Accuracy of Logistic regression model prediction on test set for MSMOTE data set: 0.80


In [30]:
# Confusion Matrix & Classification reports for the model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [31]:
# Metrics for Random undersample data set
print(confusion_matrix(y_test, pred_us))

print(classification_report(y_test, pred_us))

[[1032  251]
 [  57  160]]
              precision    recall  f1-score   support

          No       0.95      0.80      0.87      1283
         Yes       0.39      0.74      0.51       217

    accuracy                           0.79      1500
   macro avg       0.67      0.77      0.69      1500
weighted avg       0.87      0.79      0.82      1500



In [32]:
# Metrics for SMOTE data set
print(confusion_matrix(y_test, pred_smote))

print(classification_report(y_test, pred_smote))

[[1001  282]
 [  52  165]]
              precision    recall  f1-score   support

          No       0.95      0.78      0.86      1283
         Yes       0.37      0.76      0.50       217

    accuracy                           0.78      1500
   macro avg       0.66      0.77      0.68      1500
weighted avg       0.87      0.78      0.80      1500



In [33]:
# Metrics for MSMOTE data set
print(confusion_matrix(y_test, pred_msmote))

print(classification_report(y_test, pred_msmote))

[[1033  250]
 [  53  164]]
              precision    recall  f1-score   support

          No       0.95      0.81      0.87      1283
         Yes       0.40      0.76      0.52       217

    accuracy                           0.80      1500
   macro avg       0.67      0.78      0.70      1500
weighted avg       0.87      0.80      0.82      1500

